# Rustの言語機能

## ゼロコスト抽象化

- **抽象化**:
    - 複雑な仕組みのシステムの大まかな動きだけに注目できるようにすること
    - 抽象化を行うことで実装したシステムを扱いやすくすることができる
    - C++ や Java のようなオブジェクト指向言語:
        - 複雑な仕組みを持つオブジェクトの内部では、様々なデータが保持されていてデータに合わせた処理が行われている
        - 内部のデータと処理は密接な関係にあるため、この2つを紐付けてまとめ、あえてオブジェクトの外から見えなくすることで抽象化を実現する
            - これを **カプセル化** と呼ぶ
        - 処理の内容が少しだけ異なるオブジェクトがあるものの、それらの使い方が同じ場合、オブジェクトの外から見たときのインターフェイスを定め、使い方を共通化する
            - これを **ポリモーフィズム** と呼ぶ
- **ゼロコスト抽象化**:
    - 抽象化されたコード表現から具体的な動作を導くための処理を行うために、実行時に余計な負荷がかかるプログラミング言語も多い
    - ゼロコスト抽象化とは、抽象化した処理を実行するために必要な負荷を可能な限り小さくすることを意味する
        - Rust や C++ などの言語では、抽象化された表現を具体的な処理に落とし込む作業をコンパイル時に行うことで、ゼロコスト抽象化を実現している

### trait, dyn
多くのオブジェクト指向言語では、クラスを使ってカプセル化やポリモーフィズムを実現している

一方 Rust では、トレイトを使ってポリモーフィズムを実現している

- **トレイト**:
    - 様々な型に共通のメソッドを実装するように促すことができる仕組み
    - Javaのインターフェイスに似た機能を提供し、必要なメソッドや型を定義することで、共通の振る舞いを構造体や列挙体に適用する

In [2]:
/// 鳥のさえずりに関する共通の振る舞い
trait Tweet {
    fn tweet(&self); // tweetメソッドの実装が必要
    
    /// 2回 tweet するメソッド
    fn tweet_twice(&self) {
        self.tweet();
        self.tweet();
    }
    
    /// 叫ぶメソッド
    fn shout(&self) {
        println!("Uoooooooooooooooohhh!!!!");
    }
}

// => 上記トレイトを適用した型は tweet メソッドの実装を強制される

/// 鳩(Dove) 構造体
struct Dove;

/// アヒル(Duck) 構造体
struct Duck;

// Dove構造体 に Tweetトレイト を適用
impl Tweet for Dove {
    /// tweetメソッド: Coo! とさえずる
    fn tweet(&self) {
        println!("Coo!");
    }
}

// Duck構造体 に Tweetトレイト を適用
impl Tweet for Duck {
    /// tweetメソッド: Quack! とさえずる
    fn tweet(&self) {
        println!("Quack!");
    }
}

fn main() {
    let dove = Dove{};
    dove.tweet(); // => Coo!
    dove.tweet_twice(); // => Coo!\nCoo!
    dove.shout(); // => Uoooooooooooooooohhh!!!!
    
    let duck = Duck{};
    duck.tweet(); // => Quack!
    duck.tweet_twice(); // => Quack!\nQuack!
    duck.shout(); // => Uoooooooooooooooohhh!!!!
}
main();

Coo!
Coo!
Coo!
Uoooooooooooooooohhh!!!!
Quack!
Quack!
Quack!
Uoooooooooooooooohhh!!!!


一般にメソッドの呼び出し方法には、動的ディスパッチと静的ディスパッチの2つがある

- 動的ディスパッチ:
    - メソッドが呼び出されたとき、呼び出したインスタンスを動的に確認し、それに合わせた処理を実行時に定める
- 静的ディスパッチ:
    - メソッドの呼び出し元インスタンスがコンパイル時点で分かっている場合、実行する処理をコンパイル時に定める
    - コンパイル時点で処理が決まっているため、高速な処理が可能

前述のようなコードの場合、`tweet` メソッドを呼び出すインスタンスの型はコンパイル時点で分かっている（`dove` は `Dove`, `duck` は `Duck`）ため、静的ディスパッチとなる

しかし、静的ディスパッチでは解決できないシチュエーションもあり得るため、Rustでは動的ディスパッチもサポートしている

動的ディスパッチを実現するためには `dyn` キーワードを用いる

In [3]:
fn main() {
    // Tweetトレイトを適用された何かしらの型のベクタ
    // => Box<dyn Tweet>型: Tweet という共通トレイトを有するインスタンスを画一的なポインタとして扱う
    let birds: Vec<Box<dyn Tweet>> = vec![
        // Box<dyn Tweet> にまとめることで Dove も Duck も格納可能となる
        Box::new(Dove{}),
        Box::new(Duck{}),
    ];
    
    for bird in &birds {
        bird.tweet();
    }
}
main();

Coo!
Quack!


### マーカトレイト
トレイトの中にはメソッドのない、それぞれの持つ意味や役割をしるし（マーカ）のように付与するものがある

このようなトレイトをマーカトレイトと呼び、標準ライブラリでは `Copy` や `Sized` 等、以下のようなトレイトが定義されている

- `Copy`
    - 値の所有権を渡す代わりに、値のコピーを行うようにする（コピーセマンティクスの振る舞い）
        - 所有権やコピーセマンティクスについては後述
- `Send`
    - スレッド境界を越えて所有権を転送できることを示す
- `Sized`
    - メモリ上でサイズが決まっていることを示す
- `Sync`
    - スレッド間で安全に参照を共有できることを示す

これらのトレイトはデータを含んでいないため、実行時にはメモリ上にデータが存在しないが、コンパイラが安全性の検査や最適化を行う際に使用される

### ジェネリクス
あり特定の型のために用意した処理を、別の型にも用意する必要がある場合、全く同じ処理を複数用意するのは冗長である

こういった場合のために、型自体を抽象化する仕組みとしてジェネリクスがある

ジェネリクスはコンパイル時に解決され、必要な型の分だけ自動的に定義される

そのため、プログラマは抽象的な記述をでき、かつプログラムの実行は高速に行われる（ゼロコスト抽象化）

In [4]:
/// 2要素のタプルを作る関数
// どのような型の組み合わせでも処理できるように T, S というジェネリクス型を指定する
fn make_tuple<T, S>(t: T, s: S) -> (T, S) {
    (t, s)
}

// make_tuple<i32, i32>(1, 2)
let t1 = make_tuple(1, 2); // => コンパイル時に make_tuple(i32, i32) -> (i32, i32) 関数が自動定義される

// make_tuple<&str, &str>("Hello", "World")
let t1 = make_tuple("Hello", "World"); // => コンパイル時に make_tuple(&str, &str) -> (&str, &str) 関数が自動定義される

// make_tuple<i32, &str>(3, "three")
let t1 = make_tuple(3, "three"); // => コンパイル時に make_tuple(i32, &str) -> (i32, &str) 関数が自動定義される

## 所有権と借用

Rustの重要な概念として 所有権, 借用, ライフタイム というキーワードがある

Rustでは、それぞれの値に所有権があり、その所有権を持っているのは必ず1つの変数（所有者）だけである

これはつまり、全ての値が C++ で言うところの `std::unique_ptr` 型で用意されているイメージに近い

そのため、所有者である変数がスコープから外れた瞬間にその値は破棄されることになり、メモリ管理が非常に単純化されている

これにより、C/C++ のようにメモリ管理が煩雑になることを防ぎつつ、Java やその他の多くの言語に搭載されているようなガベージコレクションによる処理の遅延も防ぐことができるようになっている

一方、値の所有者が必ず1つに限られているということは、関数に値を引数で渡したりすると所有権が移ってしまい、元のルーチンに所有権を戻すのに苦労することになる

このようなとき、その値の参照を貸し出すことができ、これを **借用** と呼ぶ

つまり、所有権はもとの所有者のまま、その値を所有する権利だけ借り受けるということである

ただし、値の参照を借用している間に、所有者である変数が値を破棄してしまうと、借用者は破棄されたメモリ領域を参照することになり危険である

これを防ぐためには、参照は所有者よりも長生きであってはならず、安全に参照できる期間を明確に定める必要がある

このような安全に借用可能なきかんのことを **ライフタイム** と呼ぶ

### ムーブセマンティクス
- **セマンティクス**:
    - 意味論、意味づけのことを指す
    - シンタックス（文法、記法）と対になる言葉

多くのプログラミング言語では、ある変数と別の変数を `=` でつなげた場合、変数に紐付けられた値が右から左へコピーされる

これをコピーセマンティクスと呼び、右側の変数と左側の変数は同一の値を持つが、それぞれの値が格納されているメモリ領域は別個となる

```cpp
// C++ の場合

#include <cstdio>

struct Color {
    int r, g, b;
};

int main(int argc, char *argv[]) {
    struct Color a = {255, 255, 255};
    struct Color b = a; // 値がコピーされる => 新規に構造体 Color{255, 255, 255} が作られる
    printf("%d %d %d\n", b.r, b.g, b.b); // => 255 255 255
}
```

一方、Rustの場合はムーブセマンティクスと呼ばれる挙動となり、右側の変数が所持していた値の所有権が左側の変数に移動する

これによりデフォルトで高速な処理が可能となっている

In [5]:
struct Color {
    r: i32,
    g: i32,
    b: i32,
}

fn main() {
    let a = Color{r: 255, g: 255, b: 255};
    let b = a; // Color{r: 255, g: 255, b: 255} の所有権が a から b に移る
    
    println!("{} {} {}", b.r, b.g, b.b); // => 255 255 255
    
    // a はすでに所有権を持っていないため、以下のコードはコンパイルエラーとなる
    println!("{} {} {}", a.r, a.g, a.b);
}
main();

Error: borrow of moved value: `a`

### 借用
ある関数に引数で値を渡すとき、Rustはムーブセマンティクスであるため、値の所有権ごと関数の引数に渡すことになる

この場合、呼び出し元の処理に再び所有権を返すのは面倒である

In [6]:
fn main() {
    let mut important_data = "Hello, World!".to_string();
    
    print_data(important_data); // ここでムーブセマンティクスが起こり、important_data は所有権を失う
    println!("{}", important_data); // important_data は所有権を持っていないためコンパイルエラーとなる
}

fn print_data(data: String) {
    println!("print: {}", data);
}

main();

Error: borrow of moved value: `important_data`

In [7]:
/// 上記問題を解決するためには、呼び出し元に値の所有権を返す必要がある

fn main() {
    let mut important_data = "Hello, World!".to_string();
    
    important_data = print_data(important_data); // important_data 関数実行時に所有権を失い、戻り値として再び所有権を手に入れる
    println!("{}", important_data); // important_data は所有権を持っているためOK
}

// 所有権を返すバージョン
fn print_data(data: String) -> String {
    println!("print: {}", data);
    data
}

main();

print: Hello, World!
Hello, World!


上記のように、一旦渡したものを再び返してもらうような処理を毎回書くのは非効率的である

そこで、関数の引数には値の参照を渡すようにして、借用の概念を実現する

なお参照とは、別の変数に値の所有権を譲渡するのではなく、値へのアクセスを許可する方法である

In [8]:
fn main() {
    let mut important_data = "Hello, World!".to_string();
    
    print_data(&important_data); // important_data の参照を渡し、値へのアクセス権のみを貸し出す
    println!("{}", important_data); // important_data は所有権を持ったままのためOK
}

// 参照を借用するバージョン
fn print_data(data: &String) {
    println!("print: {}", data);
}

main();

print: Hello, World!
Hello, World!


借用を行う場合、元の変数が可変か不変かは重要である

不変な変数の参照は、いくつ参照を渡したとしても問題はない

参照先から値を変更されることはないためである

一方、可変な参照の場合、一度に渡せる参照は1つだけである

これは複数の参照が可能になった場合、複数の参照先に同時に値の更新が発生したときに、予期せず値が破壊されてしまう可能性があるためである

また、可変な参照と不変な参照を同時に存在させることもできない

不変参照の変数にとっては、可変参照の変数によって値が変更されることを予期していないためである

In [10]:
{
    let x = 5;
    let y = &x; // 不変参照1: 問題ない
    let z = &x; // 不変参照2: 問題ない

    dbg!(x);
    dbg!(y);
    dbg!(z);
}

[src/lib.rs:186] x = 5
[src/lib.rs:187] y = 5
[src/lib.rs:188] z = 5


()

In [11]:
{
    let mut x = 5;
    let y = &mut x; // 可変参照1: 問題ない
    let z = &mut x; // 可変参照2: エラー
    
    dbg!(x);
    dbg!(y);
    dbg!(z);
}

Error: cannot borrow `x` as mutable more than once at a time

Error: cannot use `x` because it was mutably borrowed

In [12]:
{
    let mut x = 5;
    let y = &x; // 不変参照1: 問題ない
    let z = &mut x; // 可変参照1: エラー
    
    dbg!(y);
    dbg!(z);
}

Error: cannot borrow `x` as mutable because it is also borrowed as immutable

### ライフタイム
変数参照の可変・不変に関わらず、参照はもとの所有者よりも長く生存することはできない

もとの所有者が値を破棄してしまった場合に、参照先が消失してしまうためである

In [13]:
{
    let y;
    {
        let x = 5; // | x のライフタイム
        y = &x;    // | | y のライフタイム
        dbg!(x);   // | |
    }              //   |
    dbg!(y);       //   |
    // => x よりも y が長く生存してはならない: エラー
}

Error: `x` does not live long enough

借用にはいくつの厳密なルールが存在し、Rustに慣れない内は謎のコンパイルエラーに悩まされることも多い

しかしながら、これらのルールはプログラミングを安全に行うためのルールであるため、慣れてさえしまえば、コンパイラによるエラーチェックにより快適なプログラミングができるようになるはずである

#### ライフタイム決定の流れ
Rustの借用チェッカーは、リソースのライフタイムを調べるために、次の2つのライフタイムに注目している

1. 参照のライフタイム
    - 参照が利用されている期間に対応したライフタイム
2. 値のライフタイム
    - 値が解放される直前までの期間に対応したライフタイム

これらのライフタイムは、制御フローをベースに特定される

In [14]:
{
    let mut x = 5;  // | x のライフタイム
    let y = &x;     // | | y のライフタイム
    let z = &mut x; // | | | z のライフタイム: 不変参照と可変参照が同時に存在するためエラー
    dbg!(y);        // | |
    dbg!(x);        // |
}

Error: unused variable: `z`

Error: cannot borrow `x` as mutable because it is also borrowed as immutable

In [15]:
{
    let mut x = 5;  // | x のライフタイム
    let y = &x;     // | | y のライフタイム
    let z = &mut x; // |   | z のライフタイム: yのライフタイムが切れているため可変参照が存在できる
    dbg!(z);        // |   |
    dbg!(x);        // |
}

[src/lib.rs:185] z = 5
[src/lib.rs:186] x = 5


()

#### ライフタイム注釈記法
ライフタイム注釈は、いかなる参照の生存期間も変えることはない

シグネチャにジェネリックな型引数を指定された関数が、あらゆる型を受け取ることができるのと同様に、ジェネリックなライフタイム引数を指定された関数は、あらゆるライフタイムの参照を受け取ることができる

ライフタイム注釈は、ライフタイムに影響することなく、複数の参照のライフタイムのお互いの関係を記述するための記法で、以下のように記述される

- ライフタイム引数の名前はアポストロフィー(`'`)で始まらなければならず、通常全て小文字で記述される
    - 基本的には、ジェネリクス型のように短い名前で命名される
    - 多くの場合、`'a` という名前が使われる
- ライフタイム引数注釈は、参照の `&` の後に配置し、注釈と参照の型を区別するために空白を1つ使用する

```rust
&i32        // (ただの)参照
&'a i32     // 明示的なライフタイム付きの参照
&'a mut i32 // 明示的なライフタイム付きの可変参照
```

1つのライフタイム注釈それだけでは、大した意味はない

注釈は、複数の参照のジェネリックなライフタイム引数が、お互いにどう関係するかをコンパイラに指示することを意図しているためである

例えば、ライフタイム `'a` 付きの i32型 への参照となる引数 `first` を受け取る関数があるとする

この関数にはさらに、`'a` のライフタイム付きの i32型 への別の参照となる `second` という別の引数もあるとする

ライフタイム注釈は、`first` と `second` の参照がどちらもこのジェネリックなライフタイムと同じだけ生きることを明示することになる

```rust
fn some_function<'a>(first: &'a i32, second: &'a i32) {
    // first と second はともに同じライフタイム 'a だけ生きる
}
```

### RAII
- **RAII**: Resource Acquisition Is Initialization
    - リソースの取得は初期化である、という意味
    - わかりやすく言えば、変数の初期化時にリソースの確保を行い、変数を破棄するときにリソースの解放を行う、ということ
        - これを正しく行わないと、メモリリーク等のリソースのリーク問題が発生することになる

Rustでは、各変数のライフタイムを厳密に管理しているため、変数がスコープから外れた瞬間に確保していたリソースは解放されることになる

このとき、「開いているファイルを閉じる」「接続しているネットワークから切断する」等の振る舞いを定義しておかないと、リソースのリーク問題が発生してしまう

これを防ぐために、Rustでは、デストラクタを定義するための `Drop` トレイトが用意されている

In [16]:
struct Droppable;

// Dropトレイト を Droppable構造体 に適用
impl Drop for Droppable {
    // dropメソッドにてデストラクタを定義可能
    fn drop(&mut self) {
        println!("Resource will be released!");
    }
}

fn main() {
    {
        let d = Droppable{};
    } // d のライフタイムが切れる => Droppable::drop メソッドが呼ばれる => Resource will be released!
    println!("The Droppable instance is dropped.");
}
main();

Resource will be released!
The Droppable instance is dropped.
